# Substitutionschiffre

In [ ]:
import numpy as np
from functools import lru_cache

from collections import defaultdict
from itertools import permutations
from urllib.request import urlopen

In [ ]:
raw = """'vk iv? nv?tz zbnpixh hr-' yb''o:b v, yba opb iv? roxxis
wb z-z evh bqbe 'hoph ibh 'v zv evh :bh bdn-hbz o;v?h -ha
- wv?tz ;b c?-hb '?pxp-'bz -, iv? wbpb o;tb hv .bbx ?x w-hr yi nrottbe:b'k
hv ;b rveb'ha - roqb ;bbe opv?ez ,vp c?-hb o wr-tb evw oez qbpi
,bw xbvxtb yozb -h ,opaaa
;b,vpb wb xpvnbbzk ,bbt ,pbb hv pboz hr-' bdnbpxh ,pvy o evqbtu


,op v?h -e hrb ?enrophbz ;on.wohbp' v, hrb ?e,o'r-veo;tb bez v,
hrb wb'hbpe 'x-pot opy v, hrb :otodi t-b' o 'yott ?epb:opzbz
ibttvw '?ea
vp;-h-e: hr-' oh o z-'hoenb v, pv?:rti e-ebhimhwv y-tt-ve y-tb' -'
oe ?hhbpti -e'-:e-,-noeh t-hhtb ;t?b :pbbe xtoebh wrv'b oxbm
zb'nbezbz t-,b ,vpy' opb 'v oyol-e:ti xp-y-h-qb hroh hrbi 'h-tt
hr-e. z-:-hot wohnrb' opb o xpbhhi eboh -zboa
hr-' xtoebh ro' m vp pohrbp roz m o xpv;tbyk wr-nr wo' hr-'u yv'h
v, hrb xbvxtb ve -h wbpb ?eroxxi ,vp xpbhhi y?nr v, hrb h-yba
yoei 'vt?h-ve' wbpb '?::b'hbz ,vp hr-' xpv;tbyk ;?h yv'h v,
hrb'b wbpb top:bti nvenbpebz w-hr hrb yvqbybeh' v, 'yott :pbbe
x-bnb' v, xoxbpk wr-nr -' vzz ;bno?'b ve hrb wrvtb -h wo'egh hrb
'yott :pbbe x-bnb' v, xoxbp hroh wbpb ?eroxxia
oez 'v hrb xpv;tby pbyo-ebzf tvh' v, hrb xbvxtb wbpb yboek oez
yv'h v, hrby wbpb y-'bpo;tbk bqbe hrb veb' w-hr z-:-hot wohnrb'a
yoei wbpb -enpbo'-e:ti v, hrb vx-e-ve hroh hrbigz ott yozb o ;-:
y-'ho.b -e nvy-e: zvwe ,pvy hrb hpbb' -e hrb ,-p'h xtonba oez
'vyb 'o-z hroh bqbe hrb hpbb' roz ;bbe o ;oz yvqbk oez hroh ev
veb 'rv?tz bqbp roqb tb,h hrb vnboe'a
oez hrbek veb hr?p'zoik ebopti hwv hrv?'oez ibop' o,hbp veb
yoe roz ;bbe eo-tbz hv o hpbb ,vp 'oi-e: rvw :pboh -h wv?tz ;b hv
;b e-nb hv xbvxtb ,vp o nroe:bk veb :-pt '-hh-e: ve rbp vwe -e o
'yott no,b -e p-n.yoe'wvphr '?zzbeti pbot-lbz wroh -h wo' hroh
roz ;bbe :v-e: wpve: ott hr-' h-ybk oez 'rb ,-eotti .ebw rvw hrb
wvptz nv?tz ;b yozb o :vvz oez roxxi xtonba hr-' h-yb -h wo'
p-:rhk -h wv?tz wvp.k oez ev veb wv?tz roqb hv :bh eo-tbz hv
oeihr-e:a
'oztik rvwbqbpk ;b,vpb 'rb nv?tz :bh hv o xrveb hv hbtt oeivebm
o;v?h -hk o hbpp-;ti 'h?x-z noho'hpvxrb vnn?ppbzk oez hrb -zbo wo'
tv'h ,vpbqbpa
hr-' -' evh rbp 'hvpia
;?h -h -' hrb 'hvpi v, hroh hbpp-;tb 'h?x-z noho'hpvxrb oez 'vyb v,
-h' nve'bc?benb'a
-h -' ot'v hrb 'hvpi v, o ;vv.k o ;vv. nottbz hrb r-hnr r-.bpg'
:?-zb hv hrb :otodi m evh oe bophr ;vv.k ebqbp x?;t-'rbz ve
bophrk oez ?eh-t hrb hbpp-;tb noho'hpvxrb vnn?ppbzk ebqbp 'bbe vp
rbopz v, ;i oei bophryoea
ebqbphrbtb''k o wrvtti pbyop.o;tb ;vv.a
-e ,onh -h wo' xpv;o;ti hrb yv'h pbyop.o;tb ;vv. bqbp hv nvyb
v?h v, hrb :pboh x?;t-'r-e: rv?'b' v, ?p'o y-evp m v, wr-nr ev
bophryoe roz bqbp rbopz b-hrbpa
evh veti -' -h o wrvtti pbyop.o;tb ;vv.k -h -' ot'v o r-:rti
'?nnb'',?t veb m yvpb xvx?top hroe hrb nbtb'h-ot rvyb nopb
vye-;?'k ;bhhbp 'btt-e: hroe ,-,hi yvpb hr-e:' hv zv -e lbpv
:poq-hik oez yvpb nvehpvqbp'-ot hroe vvtve nvtt?xr-zg' hp-tv:i v,
xr-tv'vxr-not ;tvn.;?'hbp' wrbpb :vz wbeh wpve:k 'vyb yvpb
v, :vzg' :pbohb'h y-'ho.b' oez wrv -' hr-' :vz xbp've oeiwois
-e yoei v, hrb yvpb pbtodbz n-q-t-loh-ve' ve hrb v?hbp bo'hbpe
p-y v, hrb :otodik hrb r-hnr r-.bpg' :?-zb ro' otpbozi
'?xxtoehbz hrb :pboh benintvxbz-o :otonh-no o' hrb 'hoezopz
pbxv'-hvpi v, ott .evwtbz:b oez w-'zvyk ,vp hrv?:r -h ro' yoei
vy-''-ve' oez nveho-e' y?nr hroh -' oxvnpixrotk vp oh tbo'h w-tzti
-eonn?pohbk -h 'nvpb' vqbp hrb vtzbpk yvpb xbzb'hp-oe wvp. -e hwv
-yxvphoeh pb'xbnh'a
,-p'hk -h -' 't-:rhti nrboxbpf oez 'bnvezti -h ro' hrb wvpz' zvegh
xoe-n -e'np-;bz -e top:b ,p-bezti tbhhbp' ve -h' nvqbpa
;?h hrb 'hvpi v, hr-' hbpp-;tbk 'h?x-z hr?p'zoik hrb 'hvpi v, -h'
bdhpovpz-eopi nve'bc?benb'k oez hrb 'hvpi v, rvw hrb'b
nve'bc?benb' opb -ebdhp-no;ti -ehbphw-ebz w-hr hr-' pbyop.o;tb
;vv. ;b:-e' qbpi '-yxtia
-h ;b:-e' w-hr o rv?'ba


-, iv? pbotti hr-e. iv? noe ;boh yi nrottbe:b' oez woeh hv nveh-e?bu
ho.b hrb 'yottb'h xp-yb e?y;bp w-hr hrpbb z-:-h'k hroh -' o xot-ezpvyb
oez nveho-e' hrb z-:-h ,-qba y?th-xti hroh e?y;bp ;i b-:rha
evw q-'-h hrb wb;'-hb 'hohbz ebdh oez pbxti hv hr-' yo-t
w-hr hrb to'h eoyb v, hrb xbp've hr-' oph-ntb -' o;v?ha"""

In [ ]:
@lru_cache(maxsize=10)
def fetchBook(url):
    with urlopen(url) as fd:
        text = fd.read().decode("utf-8").replace("\xa0", " ").lower()
        return " ".join(text.split("\r\n"))[1:] # Remove BOM
    
ENG = fetchBook("https://www.gutenberg.org/cache/epub/394/pg394.txt").lower()
#DEU = fetchBook("https://www.gutenberg.org/cache/epub/6343/pg6343.txt").lower()

In [ ]:
def ngrams(s, n):
    for line in s.split("\n"):
        for ngram in [line[i:(i+n)] for i in range(len(line) - n)]:
            yield ngram

In [ ]:
logLik = defaultdict(int)
total = 0
for ngram in ngrams(ENG, 4):
    logLik[ngram] += 1
    total += 1
    
for ngram in logLik.keys():
    logLik[ngram] = np.log(logLik[ngram] / total)

In [ ]:
def likelihood(text):
    ll = 0.0
    for ngram in ngrams(text, 4):
        ll += logLik[ngram]
    return ll

In [ ]:
raw[:20]

In [ ]:
abc = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
msg_alphabet = np.unique(list(ngrams(raw, 1)) + abc)
msg_alphabet

In [ ]:
xlat = dict(zip(msg_alphabet, msg_alphabet))
xlat["'"], xlat['s'] = xlat['s'], xlat["'"]
xlat["k"], xlat[','] = xlat[','], xlat["k"]

bestLL = likelihood(raw.translate(str.maketrans(xlat)))
bestXlat = xlat.copy()

In [ ]:
%%time

for i in range(5):
    xlat = dict(zip(msg_alphabet, msg_alphabet))
    xlat["'"], xlat['s'] = xlat['s'], xlat["'"]
    xlat["k"], xlat[','] = xlat[','], xlat["k"]
    stall = False
    while not stall:
        stall = True
        for p in np.random.permutation(msg_alphabet):
            for q in np.random.permutation(msg_alphabet):
                if p == q:
                    next
                xlat[p], xlat[q] = xlat[q], xlat[p]    
                ll = likelihood(raw.translate(str.maketrans(xlat)))
                if ll < bestLL:            
                    bestLL, bestXlat = ll, xlat.copy()
                    stall = False
                elif ll > bestLL:                    
                    xlat[p], xlat[q] = xlat[q], xlat[p]
    print(f"Attempt {i}: {bestLL}")

In [ ]:
plain = raw.translate(str.maketrans(bestXlat))
print(plain)

In [ ]:
N = 151
print(f"https://en.wikipedia.org/wiki/index.php?curid={N * 8}")

In [ ]:
np.sum([c == 'e' for c in plain])